<a href="https://colab.research.google.com/github/SFIComplexityExplorer/Mesa-ABM-Tutorial/blob/main/Session_9_Traders_Move_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Winati Mutmainnah (D121221014)

# Import Dependencies

In [ ]:
import mesa
import numpy as np
import math
import matplotlib.pyplot as plt

# Resource Classes

In [2]:
class Sugar(mesa.Agent):
  """
  Sugar:
  - contains an amount of sugar
  - grows 1 amount of sugar at each turn
  """

  def __init__(self, unique_id, model, pos, max_sugar):
    super().__init__(unique_id, model)
    self.pos = pos
    self.amount = max_sugar
    self.max_sugar = max_sugar


  def step(self):
    '''
    Sugar growth function, adds one unit of sugar each step until
    max amount
    '''
    self.amount = min([self.max_sugar, self.amount+1])

In [3]:
class Spice(mesa.Agent):
  """
  Spice:
  - contains an amount of spice
  - grows 1 amount of spice at each turn
  """

  def __init__(self, unique_id, model, pos, max_spice):
    super().__init__(unique_id, model)
    self.pos = pos
    self.amount = max_spice
    self.max_spice = max_spice

  def step(self):
    '''
    Spice growth function, adds one unit of spice each step until
    max amout
    '''
    self.amount = min([self.max_spice, self.amount+1])

# Trader Agent

In [ ]:
class Trader(mesa.Agent):
  """
  Trader:
  - has a metabolism of sugar and spice
  - harvest and trade sugar and spice to survive
  """

  def __init__(self, unique_id, model, pos, moore=False, sugar=0,
               spice=0, metabolism_sugar=0, metabolism_spice=0,
               vision=0):
    super().__init__(unique_id, model)
    self.pos = pos
    self.moore = moore
    self.sugar = sugar
    self.spice = spice
    self.metabolism_sugar = metabolism_sugar
    self.metabolism_spice = metabolism_spice
    self.vision = vision

    def move(self):
        print(f"I am agent {self.unique_id} and am about to move")



# Model Class

In [10]:
class SugarscapeG1mt(mesa.Model):
    """
    Manager class to run Sugarscape with Traders
    """

    def __init__(self, width=50, height=50, initial_population=200,
                 endowment_min=25, endowment_max=50, metabolism_min=1,
                 metabolism_max=5, vision_min=1, vision_max=5):

        # Initialize the base Model class
        super().__init__()

        # Initialize width and height of sugarscape
        self.width = width
        self.height = height

        # Initialize population attributes
        self.initial_population = initial_population
        self.endowment_min = endowment_min
        self.endowment_max = endowment_max
        self.metabolism_min = metabolism_min
        self.metabolism_max = metabolism_max
        self.vision_min = vision_min
        self.vision_max = vision_max

        # Initiate activation schedule
        self.schedule = mesa.time.RandomActivationByType(self)
        # Initiate mesa grid class
        self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

        # Read in landscape file from supplementary material
        sugar_distribution = np.genfromtxt("sugar-map.txt")
        spice_distribution = np.flip(sugar_distribution, 1)

        agent_id = 0
        for _, (x, y) in self.grid.coord_iter():
            max_sugar = sugar_distribution[x, y]
            if max_sugar > 0:
                sugar = Sugar(agent_id, self, (x, y), max_sugar)
                self.schedule.add(sugar)
                self.grid.place_agent(sugar, (x, y))
                agent_id += 1

            max_spice = spice_distribution[x, y]
            if max_spice > 0:
                spice = Spice(agent_id, self, (x, y), max_spice)
                self.schedule.add(spice)
                self.grid.place_agent(spice, (x, y))
                agent_id += 1

        for i in range(self.initial_population):
            # Get agent position
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            # Give agents initial endowment
            sugar = int(self.random.uniform(self.endowment_min, self.endowment_max + 1))
            spice = int(self.random.uniform(self.endowment_min, self.endowment_max + 1))
            # Give agents initial metabolism
            metabolism_sugar = int(self.random.uniform(self.metabolism_min, self.metabolism_max + 1))
            metabolism_spice = int(self.random.uniform(self.metabolism_min, self.metabolism_max + 1))
            # Give agents vision
            vision = int(self.random.uniform(self.vision_min, self.vision_max + 1))
            # Create Trader object
            trader = Trader(agent_id,
                            self,
                            (x, y),
                            moore=False,
                            sugar=sugar,
                            spice=spice,
                            metabolism_sugar=metabolism_sugar,
                            metabolism_spice=metabolism_spice,
                            vision=vision)
            # Place agent
            self.grid.place_agent(trader, (x, y))
            self.schedule.add(trader)
            agent_id += 1

    def step(self):
        '''
        Unique step function that does staged activation of sugar and spice
        and then randomly activates traders
        '''
        # Step Sugar agents
        for sugar in self.schedule.agents_by_type[Sugar].values():
            sugar.step()

        # Step Spice agents
        for spice in self.schedule.agents_by_type[Spice].values():
            spice.step()

        # Step Trader agents
        trader_shuffle = list(self.schedule.agents_by_type[Trader].values())
        self.random.shuffle(trader_shuffle)

        for agent in trader_shuffle:
            agent.move()

        self.schedule.steps += 1  # Important for data collector to track number of steps

    def run_model(self, step_count=1000):
        for i in range(step_count):
            self.step()

# Run Sugarscape

In [ ]:
model = SugarscapeG1mt()
model.run_model(step_count=1)

I am agent 4210 and am about to move
I am agent 4285 and am about to move
I am agent 4226 and am about to move
I am agent 4289 and am about to move
I am agent 4267 and am about to move
I am agent 4244 and am about to move
I am agent 4328 and am about to move
I am agent 4259 and am about to move
I am agent 4266 and am about to move
I am agent 4151 and am about to move
I am agent 4190 and am about to move
I am agent 4318 and am about to move
I am agent 4160 and am about to move
I am agent 4138 and am about to move
I am agent 4175 and am about to move
I am agent 4144 and am about to move
I am agent 4245 and am about to move
I am agent 4284 and am about to move
I am agent 4273 and am about to move
I am agent 4188 and am about to move
I am agent 4164 and am about to move
I am agent 4173 and am about to move
I am agent 4247 and am about to move
I am agent 4256 and am about to move
I am agent 4208 and am about to move
I am agent 4322 and am about to move
I am agent 4196 and am about to move
I

C:\Users\lenovo\AppData\Local\Temp\ipykernel_13356\2607841188.py:48: UserWarning: Agent 0 is being placed with
place_agent() despite already having the position (0, 0). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(spice, (x, y))
C:\Users\lenovo\AppData\Local\Temp\ipykernel_13356\2607841188.py:48: UserWarning: Agent 1 is being placed with
place_agent() despite already having the position (0, 1). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(spice, (x, y))
C:\Users\lenovo\AppData\Local\Temp\ipykernel_13356\2607841188.py:48: UserWarning: Agent 2 is being placed with
place_agent() despite already having the position (0, 2). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(spice, (x, y))
C:\Users\lenovo\AppData\Local\Temp\ipykernel_13356\2